In [223]:
import pandas as pd
import numpy as np
import keras_contrib
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import LSTM, Dense, TimeDistributed, Embedding, Bidirectional
from keras.models import Model, Input
from keras_contrib.layers import CRF
from keras.callbacks import ModelCheckpoint
import csv
import warnings
warnings.filterwarnings("ignore")
import random
random.seed(10)
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn_crfsuite.metrics import flat_classification_report
from sklearn.metrics import f1_score
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report
from keras.preprocessing.text import text_to_word_sequence

In [224]:
tsvfile=open("NCBI-disease-IOB/train.tsv",encoding="utf-8")
tsvreader = csv.reader(tsvfile, delimiter="\t")
data=[]
sentences=[]
words=[]
tags=[]
maxsentlen=0
for line in tsvreader:
    if line!=[]:
        data.append(line)
        if line[0] not in words:
            words.append(line[0])
        if line[1] not in tags:
            tags.append(line[1])
    else:
        sentences.append(data)
        if maxsentlen<len(data):
            maxsentlen=len(data)
        data=[]

In [225]:
print(len(sentences))
print(len(words))

5422
9307


In [226]:
word_to_index = {w : i + 2 for i, w in enumerate(words)}
print(word_to_index[words[0]])
word_to_index["UNK"] = 1
word_to_index["PAD"] = 0

# Dictionary lable:index pair
# label is key and value is index.
tag_to_index = {t : i + 1 for i, t in enumerate(tags)}
print(tag_to_index[tags[0]])

tag_to_index["PAD"] = 0

idx2word = {i: w for w, i in word_to_index.items()}
print(idx2word[2])
idx2tag = {i: w for w, i in tag_to_index.items()}
print(len(tag_to_index))

2
1
Identification
4


In [354]:
# Number of data points passed in each iteration
batch_size =50
# Passes through entire dataset
epochs = 30
# Maximum length of review
max_len = maxsentlen+15
# Dimension of embedding vector
embedding = 200

In [355]:
X = [[word_to_index[w[0]] for w in s] for s in sentences]
print(X[0],X[1])
count=0
for s in sentences:
    print(s)
    count=count+1
    if count>5:
        break
# Padding each sequence to have same length  of each word
X = pad_sequences(maxlen = max_len, sequences = X, padding = "post", value = word_to_index["PAD"])

[2, 3, 4, 5, 6, 7, 3, 8, 9, 10, 11, 12, 13, 14] [15, 9, 10, 11, 16, 17, 18, 12, 19, 13, 20, 21, 8, 22, 23, 24, 25, 26, 6, 27, 28, 29, 30, 31, 32, 16, 33, 19, 32, 18, 5, 34, 35, 36, 37, 38, 14]
[['Identification', 'O'], ['of', 'O'], ['APC2', 'O'], [',', 'O'], ['a', 'O'], ['homologue', 'O'], ['of', 'O'], ['the', 'O'], ['adenomatous', 'B-Disease'], ['polyposis', 'I-Disease'], ['coli', 'I-Disease'], ['tumour', 'I-Disease'], ['suppressor', 'O'], ['.', 'O']]
[['The', 'O'], ['adenomatous', 'B-Disease'], ['polyposis', 'I-Disease'], ['coli', 'I-Disease'], ['(', 'I-Disease'], ['APC', 'I-Disease'], [')', 'I-Disease'], ['tumour', 'I-Disease'], ['-', 'O'], ['suppressor', 'O'], ['protein', 'O'], ['controls', 'O'], ['the', 'O'], ['Wnt', 'O'], ['signalling', 'O'], ['pathway', 'O'], ['by', 'O'], ['forming', 'O'], ['a', 'O'], ['complex', 'O'], ['with', 'O'], ['glycogen', 'O'], ['synthase', 'O'], ['kinase', 'O'], ['3beta', 'O'], ['(', 'O'], ['GSK', 'O'], ['-', 'O'], ['3beta', 'O'], [')', 'O'], [',', 'O']

In [356]:
print(X[0])
len(X[0])

[ 2  3  4  5  6  7  3  8  9 10 11 12 13 14  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]


138

In [357]:
# Convert label to index
y = [[tag_to_index[w[1]] for w in s] for s in sentences]
print(y[0])
# padding
y = pad_sequences(maxlen = max_len, sequences = y, padding = "post", value = tag_to_index["PAD"])

[1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 3, 3, 1, 1]


In [358]:
num_tag = 3
print(num_tag)
# One hot encoded labels
y = [to_categorical(i, num_classes = num_tag + 1) for i in y]
print(y[0][1],y[0][18])
print(len(y[0][1]))

3
[0. 1. 0. 0.] [1. 0. 0. 0.]
4


In [361]:
# Let's check the first sentence before and after processing.
print('*****Before Processing first sentence : *****\n', ' '.join([w[0] for w in sentences[0]]))
print('*****After Processing first sentence : *****\n ', X[0])
len(X[0])

*****Before Processing first sentence : *****
 Identification of APC2 , a homologue of the adenomatous polyposis coli tumour suppressor .
*****After Processing first sentence : *****
  [ 2  3  4  5  6  7  3  8  9 10 11 12 13 14  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]


138

In [362]:
# First label before and after processing.
print('*****Before Processing first sentence : *****\n', ' '.join([w[1] for w in sentences[0]]))
print('*****After Processing first sentence : *****\n ', y[0])

*****Before Processing first sentence : *****
 O O O O O O O O B-Disease I-Disease I-Disease I-Disease O O
*****After Processing first sentence : *****
  [[0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0

In [ ]:
num_tags = 3
# Model architecture
input = Input(shape = (max_len,))
model = Embedding(input_dim = len(words) + 2, output_dim = embedding, input_length = max_len, mask_zero = True)(input)
model = Bidirectional(LSTM(units = 100, return_sequences=True, recurrent_dropout=0.1))(model)
crf = CRF(num_tags+1)  # CRF layer
out = crf(model)  # output

model = Model(input, out)
model.compile(optimizer="rmsprop", loss=crf.loss_function, metrics=[crf.accuracy])

#model.summary()

In [364]:
checkpointer = ModelCheckpoint(filepath = 'model.h5',
                       verbose = 0,
                       mode = 'auto',
                       save_best_only = True,
                       monitor='val_loss')

In [ ]:
history = model.fit(X, np.array(y), batch_size=batch_size, epochs=epochs,
                    validation_split=0.1)

Train on 4879 samples, validate on 543 samples
Epoch 1/30
4879/4879 [==============================] - ETA: 15:17 - loss: 13.3375 - crf_viterbi_accuracy: 0.0000e+ - ETA: 8:28 - loss: 12.3250 - crf_viterbi_accuracy: 0.0000e+00 - ETA: 6:11 - loss: 11.3888 - crf_viterbi_accuracy: 0.0000e+0 - ETA: 5:02 - loss: 11.4253 - crf_viterbi_accuracy: 0.0802    - ETA: 4:19 - loss: 11.0716 - crf_viterbi_accuracy: 0.181 - ETA: 3:51 - loss: 10.9377 - crf_viterbi_accuracy: 0.289 - ETA: 3:31 - loss: 10.9951 - crf_viterbi_accuracy: 0.371 - ETA: 3:15 - loss: 10.9116 - crf_viterbi_accuracy: 0.435 - ETA: 3:02 - loss: 10.8581 - crf_viterbi_accuracy: 0.487 - ETA: 2:52 - loss: 10.6174 - crf_viterbi_accuracy: 0.529 - ETA: 2:43 - loss: 10.6835 - crf_viterbi_accuracy: 0.564 - ETA: 2:36 - loss: 10.4601 - crf_viterbi_accuracy: 0.591 - ETA: 2:30 - loss: 10.2967 - crf_viterbi_accuracy: 0.616 - ETA: 2:24 - loss: 10.4570 - crf_viterbi_accuracy: 0.639 - ETA: 2:19 - loss: 10.3665 - crf_viterbi_accuracy: 0.657 - ETA: 2:15 

4879/4879 [==============================] - ETA: 2:14 - loss: 8.9727 - crf_viterbi_accuracy: 0.97 - ETA: 2:06 - loss: 8.7320 - crf_viterbi_accuracy: 0.97 - ETA: 2:03 - loss: 8.3488 - crf_viterbi_accuracy: 0.97 - ETA: 2:01 - loss: 8.5350 - crf_viterbi_accuracy: 0.97 - ETA: 1:59 - loss: 8.5292 - crf_viterbi_accuracy: 0.97 - ETA: 1:59 - loss: 8.8004 - crf_viterbi_accuracy: 0.97 - ETA: 1:58 - loss: 8.6126 - crf_viterbi_accuracy: 0.97 - ETA: 1:57 - loss: 8.5946 - crf_viterbi_accuracy: 0.97 - ETA: 1:56 - loss: 8.7341 - crf_viterbi_accuracy: 0.97 - ETA: 1:54 - loss: 8.7089 - crf_viterbi_accuracy: 0.97 - ETA: 1:53 - loss: 8.8215 - crf_viterbi_accuracy: 0.97 - ETA: 1:53 - loss: 8.9176 - crf_viterbi_accuracy: 0.97 - ETA: 1:51 - loss: 8.8645 - crf_viterbi_accuracy: 0.97 - ETA: 1:50 - loss: 8.9463 - crf_viterbi_accuracy: 0.97 - ETA: 1:49 - loss: 9.0436 - crf_viterbi_accuracy: 0.97 - ETA: 1:47 - loss: 9.0590 - crf_viterbi_accuracy: 0.97 - ETA: 1:46 - loss: 9.0641 - crf_viterbi_accuracy: 0.97 - ETA

4879/4879 [==============================] - ETA: 2:08 - loss: 8.1646 - crf_viterbi_accuracy: 0.97 - ETA: 2:05 - loss: 8.5052 - crf_viterbi_accuracy: 0.98 - ETA: 2:03 - loss: 8.6609 - crf_viterbi_accuracy: 0.98 - ETA: 2:01 - loss: 8.4240 - crf_viterbi_accuracy: 0.98 - ETA: 2:00 - loss: 9.1096 - crf_viterbi_accuracy: 0.98 - ETA: 1:58 - loss: 9.4170 - crf_viterbi_accuracy: 0.98 - ETA: 1:58 - loss: 9.4550 - crf_viterbi_accuracy: 0.98 - ETA: 1:56 - loss: 9.4279 - crf_viterbi_accuracy: 0.98 - ETA: 1:55 - loss: 9.6207 - crf_viterbi_accuracy: 0.98 - ETA: 1:54 - loss: 9.4630 - crf_viterbi_accuracy: 0.98 - ETA: 1:52 - loss: 9.3344 - crf_viterbi_accuracy: 0.98 - ETA: 1:51 - loss: 9.2660 - crf_viterbi_accuracy: 0.98 - ETA: 1:49 - loss: 9.1147 - crf_viterbi_accuracy: 0.98 - ETA: 1:48 - loss: 9.1426 - crf_viterbi_accuracy: 0.98 - ETA: 1:47 - loss: 9.1832 - crf_viterbi_accuracy: 0.98 - ETA: 1:46 - loss: 9.2724 - crf_viterbi_accuracy: 0.98 - ETA: 1:44 - loss: 9.2055 - crf_viterbi_accuracy: 0.98 - ETA

4879/4879 [==============================] - ETA: 2:06 - loss: 10.4808 - crf_viterbi_accuracy: 0.993 - ETA: 2:07 - loss: 10.5480 - crf_viterbi_accuracy: 0.996 - ETA: 2:04 - loss: 10.4789 - crf_viterbi_accuracy: 0.991 - ETA: 2:03 - loss: 10.0786 - crf_viterbi_accuracy: 0.992 - ETA: 2:00 - loss: 9.8478 - crf_viterbi_accuracy: 0.991 - ETA: 1:59 - loss: 9.9729 - crf_viterbi_accuracy: 0.99 - ETA: 1:57 - loss: 9.6261 - crf_viterbi_accuracy: 0.99 - ETA: 1:56 - loss: 9.3670 - crf_viterbi_accuracy: 0.99 - ETA: 1:55 - loss: 9.3271 - crf_viterbi_accuracy: 0.99 - ETA: 1:54 - loss: 9.8418 - crf_viterbi_accuracy: 0.99 - ETA: 1:52 - loss: 9.9113 - crf_viterbi_accuracy: 0.99 - ETA: 1:51 - loss: 9.8443 - crf_viterbi_accuracy: 0.99 - ETA: 1:49 - loss: 9.6889 - crf_viterbi_accuracy: 0.99 - ETA: 1:48 - loss: 9.6765 - crf_viterbi_accuracy: 0.99 - ETA: 1:47 - loss: 9.7417 - crf_viterbi_accuracy: 0.99 - ETA: 1:45 - loss: 9.8388 - crf_viterbi_accuracy: 0.99 - ETA: 1:44 - loss: 9.7118 - crf_viterbi_accuracy: 0

4879/4879 [==============================] - ETA: 2:08 - loss: 10.5602 - crf_viterbi_accuracy: 0.993 - ETA: 2:06 - loss: 9.6390 - crf_viterbi_accuracy: 0.995 - ETA: 2:05 - loss: 9.2194 - crf_viterbi_accuracy: 0.99 - ETA: 2:03 - loss: 8.7662 - crf_viterbi_accuracy: 0.99 - ETA: 2:01 - loss: 9.1418 - crf_viterbi_accuracy: 0.99 - ETA: 1:59 - loss: 9.2547 - crf_viterbi_accuracy: 0.99 - ETA: 1:57 - loss: 9.0217 - crf_viterbi_accuracy: 0.99 - ETA: 1:56 - loss: 8.9552 - crf_viterbi_accuracy: 0.99 - ETA: 1:54 - loss: 8.9346 - crf_viterbi_accuracy: 0.99 - ETA: 1:53 - loss: 9.2099 - crf_viterbi_accuracy: 0.99 - ETA: 1:53 - loss: 9.1903 - crf_viterbi_accuracy: 0.99 - ETA: 1:51 - loss: 9.1837 - crf_viterbi_accuracy: 0.99 - ETA: 1:50 - loss: 9.3755 - crf_viterbi_accuracy: 0.99 - ETA: 1:48 - loss: 9.2477 - crf_viterbi_accuracy: 0.99 - ETA: 1:47 - loss: 9.3251 - crf_viterbi_accuracy: 0.99 - ETA: 1:46 - loss: 9.3138 - crf_viterbi_accuracy: 0.99 - ETA: 1:44 - loss: 9.2968 - crf_viterbi_accuracy: 0.99 - 

...

4879/4879 [==============================] - ETA: 2:02 - loss: 9.7849 - crf_viterbi_accuracy: 1.00 - ETA: 2:03 - loss: 9.4979 - crf_viterbi_accuracy: 0.99 - ETA: 2:05 - loss: 9.9361 - crf_viterbi_accuracy: 0.99 - ETA: 2:01 - loss: 9.1716 - crf_viterbi_accuracy: 0.99 - ETA: 2:00 - loss: 9.0877 - crf_viterbi_accuracy: 0.99 - ETA: 1:59 - loss: 9.2925 - crf_viterbi_accuracy: 0.99 - ETA: 1:58 - loss: 9.1966 - crf_viterbi_accuracy: 0.99 - ETA: 1:57 - loss: 9.1707 - crf_viterbi_accuracy: 0.99 - ETA: 1:55 - loss: 9.5825 - crf_viterbi_accuracy: 0.99 - ETA: 1:53 - loss: 9.7127 - crf_viterbi_accuracy: 0.99 - ETA: 1:52 - loss: 9.6381 - crf_viterbi_accuracy: 0.99 - ETA: 1:50 - loss: 9.5181 - crf_viterbi_accuracy: 0.99 - ETA: 1:49 - loss: 9.3561 - crf_viterbi_accuracy: 0.99 - ETA: 1:48 - loss: 9.2906 - crf_viterbi_accuracy: 0.99 - ETA: 1:46 - loss: 9.1830 - crf_viterbi_accuracy: 0.99 - ETA: 1:45 - loss: 9.2349 - crf_viterbi_accuracy: 0.99 - ETA: 1:43 - loss: 9.1895 - crf_viterbi_accuracy: 0.99 - ETA

4879/4879 [==============================] - ETA: 2:08 - loss: 8.7089 - crf_viterbi_accuracy: 1.00 - ETA: 2:04 - loss: 10.5727 - crf_viterbi_accuracy: 1.000 - ETA: 2:03 - loss: 9.9832 - crf_viterbi_accuracy: 1.000 - ETA: 2:00 - loss: 10.5595 - crf_viterbi_accuracy: 1.000 - ETA: 1:59 - loss: 10.1302 - crf_viterbi_accuracy: 1.000 - ETA: 1:58 - loss: 10.1064 - crf_viterbi_accuracy: 1.000 - ETA: 1:56 - loss: 10.1080 - crf_viterbi_accuracy: 1.000 - ETA: 1:55 - loss: 9.8771 - crf_viterbi_accuracy: 1.000 - ETA: 1:54 - loss: 9.9959 - crf_viterbi_accuracy: 1.00 - ETA: 1:53 - loss: 10.1735 - crf_viterbi_accuracy: 1.000 - ETA: 1:52 - loss: 10.2932 - crf_viterbi_accuracy: 0.999 - ETA: 1:51 - loss: 10.3310 - crf_viterbi_accuracy: 0.999 - ETA: 1:50 - loss: 10.1963 - crf_viterbi_accuracy: 0.999 - ETA: 1:49 - loss: 10.0285 - crf_viterbi_accuracy: 0.999 - ETA: 1:47 - loss: 9.9740 - crf_viterbi_accuracy: 0.999 - ETA: 1:46 - loss: 9.8966 - crf_viterbi_accuracy: 0.99 - ETA: 1:45 - loss: 9.7550 - crf_viter

4879/4879 [==============================] - ETA: 2:08 - loss: 9.9576 - crf_viterbi_accuracy: 1.00 - ETA: 2:06 - loss: 10.4481 - crf_viterbi_accuracy: 1.000 - ETA: 2:05 - loss: 9.5494 - crf_viterbi_accuracy: 1.000 - ETA: 2:04 - loss: 9.2059 - crf_viterbi_accuracy: 1.00 - ETA: 2:02 - loss: 8.9464 - crf_viterbi_accuracy: 1.00 - ETA: 2:00 - loss: 8.6800 - crf_viterbi_accuracy: 1.00 - ETA: 1:59 - loss: 8.5416 - crf_viterbi_accuracy: 1.00 - ETA: 1:58 - loss: 8.8059 - crf_viterbi_accuracy: 1.00 - ETA: 1:57 - loss: 9.1244 - crf_viterbi_accuracy: 1.00 - ETA: 1:55 - loss: 9.0722 - crf_viterbi_accuracy: 0.99 - ETA: 1:54 - loss: 9.0085 - crf_viterbi_accuracy: 0.99 - ETA: 1:52 - loss: 8.9376 - crf_viterbi_accuracy: 0.99 - ETA: 1:50 - loss: 8.8324 - crf_viterbi_accuracy: 0.99 - ETA: 1:49 - loss: 8.8724 - crf_viterbi_accuracy: 0.99 - ETA: 1:48 - loss: 8.8685 - crf_viterbi_accuracy: 0.99 - ETA: 1:47 - loss: 8.9101 - crf_viterbi_accuracy: 0.99 - ETA: 1:46 - loss: 8.8897 - crf_viterbi_accuracy: 0.99 - 

4879/4879 [==============================] - ETA: 2:19 - loss: 9.8995 - crf_viterbi_accuracy: 1.00 - ETA: 2:13 - loss: 10.3438 - crf_viterbi_accuracy: 1.000 - ETA: 2:10 - loss: 9.1283 - crf_viterbi_accuracy: 1.000 - ETA: 2:07 - loss: 10.0136 - crf_viterbi_accuracy: 1.000 - ETA: 2:04 - loss: 9.5924 - crf_viterbi_accuracy: 1.000 - ETA: 2:02 - loss: 9.6984 - crf_viterbi_accuracy: 1.00 - ETA: 2:00 - loss: 9.8422 - crf_viterbi_accuracy: 1.00 - ETA: 1:58 - loss: 9.6492 - crf_viterbi_accuracy: 1.00 - ETA: 1:57 - loss: 9.5502 - crf_viterbi_accuracy: 1.00 - ETA: 1:56 - loss: 9.5284 - crf_viterbi_accuracy: 1.00 - ETA: 1:55 - loss: 9.7076 - crf_viterbi_accuracy: 1.00 - ETA: 1:54 - loss: 9.5351 - crf_viterbi_accuracy: 1.00 - ETA: 1:53 - loss: 9.7027 - crf_viterbi_accuracy: 1.00 - ETA: 1:51 - loss: 9.6043 - crf_viterbi_accuracy: 1.00 - ETA: 1:50 - loss: 9.6457 - crf_viterbi_accuracy: 1.00 - ETA: 1:49 - loss: 9.7459 - crf_viterbi_accuracy: 1.00 - ETA: 1:48 - loss: 9.6779 - crf_viterbi_accuracy: 1.00

4879/4879 [==============================] - ETA: 2:01 - loss: 9.5488 - crf_viterbi_accuracy: 1.00 - ETA: 2:00 - loss: 10.2323 - crf_viterbi_accuracy: 1.000 - ETA: 2:00 - loss: 9.9822 - crf_viterbi_accuracy: 1.000 - ETA: 1:58 - loss: 9.9088 - crf_viterbi_accuracy: 1.00 - ETA: 1:57 - loss: 10.0645 - crf_viterbi_accuracy: 1.000 - ETA: 1:55 - loss: 9.5778 - crf_viterbi_accuracy: 1.000 - ETA: 1:55 - loss: 9.4695 - crf_viterbi_accuracy: 1.00 - ETA: 1:53 - loss: 9.5477 - crf_viterbi_accuracy: 1.00 - ETA: 1:52 - loss: 9.5811 - crf_viterbi_accuracy: 1.00 - ETA: 1:51 - loss: 9.9943 - crf_viterbi_accuracy: 0.99 - ETA: 1:50 - loss: 9.9706 - crf_viterbi_accuracy: 0.99 - ETA: 1:48 - loss: 9.8487 - crf_viterbi_accuracy: 0.99 - ETA: 1:47 - loss: 10.0127 - crf_viterbi_accuracy: 0.999 - ETA: 1:46 - loss: 9.8855 - crf_viterbi_accuracy: 0.999 - ETA: 1:45 - loss: 9.7679 - crf_viterbi_accuracy: 0.99 - ETA: 1:44 - loss: 9.7182 - crf_viterbi_accuracy: 0.99 - ETA: 1:43 - loss: 9.6308 - crf_viterbi_accuracy: 0

In [ ]:
acc = history.history['crf_viterbi_accuracy']
val_acc = history.history['val_crf_viterbi_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
plt.figure(figsize = (8, 8))
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

In [ ]:
plt.figure(figsize = (8, 8))
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
tsvfile=open("NCBI-disease-IOB/test.tsv",encoding="utf-8")
tsvreader = csv.reader(tsvfile, delimiter="\t")
data1=[]
sentences1=[]
words1=[]
tags1=[]
for line in tsvreader:
    if line!=[]:
        data1.append(line)
        if line[0] not in words1:
            words1.append(line[0])
        if line[1] not in tags1:
            tags1.append(line[1])
    else:
        sentences1.append(data1)
        data1=[]

X1 = [[word_to_index[w[0] if w[0] in words else "UNK"] for w in s] for s in sentences1]
print(X1[0],X1[1])
count=0
for s in sentences1:
    print(s)
    count=count+1
    if count>5:
        break
# Padding each sequence to have same length  of each word
X1 = pad_sequences(maxlen = max_len, sequences = X1, padding = "post", value = word_to_index["PAD"])
y1 = [[tag_to_index[w[1]] for w in s] for s in sentences1]
print(y1[0])
# padding
y1 = pad_sequences(maxlen = max_len, sequences = y1, padding = "post", value = tag_to_index["PAD"])
num_tag = 3
print(num_tag)
# One hot encoded labels
y1=[to_categorical(i, num_classes = num_tag + 1) for i in y1]

In [ ]:
y_pred = model.predict(X1)
y_pred = np.argmax(y_pred, axis=-1)
y_test_true = np.argmax(y1, -1)

In [ ]:
y_pred = [[idx2tag[i] for i in row] for row in y_pred]
y_test_true = [[idx2tag[i] for i in row] for row in y_test_true] 

In [ ]:
print("F1-score is : {:.1%}".format(f1_score(y_test_true, y_pred)))

In [ ]:
report = flat_classification_report(y_pred=y_pred, y_true=y_test_true)
print(report)

In [ ]:
TP=0
TN=0
FP=0
FN=0
i=0
nument=0
while i<len(y_test_true):
    j=0
    #print(i)
    while j <len(y_test_true[i]):
        #print('j ',j)
        if y_test_true[i][j]=='B-Disease':
            st=j
            nument=nument+1
            j=j+1
            while (y_test_true[i][j]=='I-Disease' or y_test_true[i][j]=='E-Disease') and j<len(y_test_true[i]):
                j=j+1
                #print('inside ',j)
            end=j
            if y_test_true[i][st:end]==y_pred[i][st:end]:
                TP=TP+1
            else:
                FN=FN+1
        elif y_test_true[i][j]=='O' and y_pred[i][j]=='O':
            TN=TN+1
            
        elif y_test_true[i][j]=='O' and y_pred[i][j]!='O':
            FP=FP+1
            
        j=j+1
        
    i=i+1
    
Precision=TP/(TP+FP)
print(Precision)
Recall=TP/(TP+FN)
print(Recall)
F1=(2*Precision*Recall)/(Precision+Recall)
print(F1)
mat=[[TP,FP],[FN,TN]]
print(mat)

In [ ]:
TP=0
TN=0
FP=0
FN=0
i=0
nument=0
while i<len(y_test_true):
    j=0
    #print(i)
    while j <len(y_test_true[i]):
        #print('j ',j)
        if y_test_true[i][j]=='B-Disease':
            st=j
            nument=nument+1
            j=j+1
            while (y_test_true[i][j]=='I-Disease' or y_test_true[i][j]=='E-Disease') and j<len(y_test_true[i]):
                j=j+1
                #print('inside ',j)
            end=j
            found=0
            for k in range(st,end):
                if y_test_true[i][k]==y_pred[i][k]:
                    found=1
                    break
            if found==1:
                TP=TP+1
            else:
                FN=FN+1
            '''if y_test_true[i][st:end]==y_pred[i][st:end]:
                TP=TP+1'''
            
        elif y_test_true[i][j]=='O' and y_pred[i][j]=='O':
            TN=TN+1
            
        elif y_test_true[i][j]=='O' and y_pred[i][j]!='O':
            FP=FP+1
            
        j=j+1
        
    i=i+1
    
Precision=TP/(TP+FP)
print(Precision)
Recall=TP/(TP+FN)
print(Recall)
F1=(2*Precision*Recall)/(Precision+Recall)
print(F1)
mat=[[TP,FP],[FN,TN]]
print(mat)